In [25]:
import torch

class Channel:
    def __init__(self, name="no_name", masses=None, couplings=None, cheby_coeffs=None):
        """
        Initialize a Channel with optional name, masses, couplings, and Chebyshev coefficients.
        - masses: list of particle masses for this channel.
        - couplings: list of coupling constants for resonances (one per resonance).
        - cheby_coeffs: list of Chebyshev coefficients for form-factor numerator expansion.
        """
        if masses is None:
            masses = []
        if couplings is None:
            couplings = []
        if cheby_coeffs is None:
            cheby_coeffs = []
        self.channel_name = name
        self.masses = list(masses)               # list of particle masses (floats)
        self.couplings = list(couplings)         # list of coupling strengths (floats)
        self.couplings_steps = []                # optional step adjustments for couplings
        self.chebyCoefficients = list(cheby_coeffs)   # Chebyshev expansion coefficients (floats)
        self.chebyCoeff_steps = []               # optional step adjustments for Chebyshev coeffs
        self.poletype = 1    # integer indicating pole type (used for choosing omega mapping)
        self.s0 = 1.0        # Chebyshev expansion scale parameter (can be channel-specific)
    
    def getMomentum(self, s):
        """Compute the channel momentum p = 0.5 * sqrt(s - (sum(masses))^2)."""
        # Convert input s to a torch tensor (complex128) for calculation
        s_t = s if torch.is_tensor(s) else torch.tensor(s, dtype=torch.complex128)
        # Sum all masses (as float) and compute threshold mass sum
        mass_sum = sum(self.masses)
        # Use complex sqrt for below-threshold (s_t - mass_sum**2 may be negative or complex)
        p = 0.5 * torch.sqrt(s_t - (mass_sum ** 2))
        return p

    def getComplexMomentum(self, s):
        """Compute alternative momentum definition: p_complex = 0.5 * sqrt(-s + (sum(masses))^2)."""
        s_t = s if torch.is_tensor(s) else torch.tensor(s, dtype=torch.complex128)
        mass_sum = sum(self.masses)
        p_c = 0.5 * torch.sqrt(-s_t + (mass_sum ** 2))
        return p_c

    def getTrueMomentum(self, s):
        """Compute the physical two-body momentum: 
           p_true = 0.5 * sqrt((s - (m1+m2)^2)*(s - (m1-m2)^2)) / s, for two-body channels."""
        s_t = s if torch.is_tensor(s) else torch.tensor(s, dtype=torch.complex128)
        if len(self.masses) < 2:
            # If we don't have two masses, fall back to simple momentum definition
            return self.getMomentum(s_t)
        m1, m2 = self.masses[0], self.masses[1]
        # Compute (m1+m2)^2 and (m1-m2)^2
        sum_sq = (m1 + m2) ** 2
        diff_sq = (m1 - m2) ** 2
        # Compute sqrt((s - sum_sq) * (s - diff_sq)) and divide by s
        p_true = 0.5 * torch.sqrt((s_t - sum_sq) * (s_t - diff_sq)) / s_t
        return p_true

    def getThreshold(self):
        """Return the channel threshold in terms of s (i.e., (sum of masses)^2)."""
        mass_sum = sum(self.masses)
        return mass_sum ** 2

    # Coupling and Chebyshev coefficient management
    def getCouplings(self):
        """Get the list of coupling constants for this channel."""
        return list(self.couplings)

    def setCouplings(self, c_list):
        """Set the list of coupling constants for this channel."""
        self.couplings = list(c_list)

    def getCoupling(self, i):
        """Get the i-th coupling constant."""
        return self.couplings[i]

    def addCoupling(self, coup, step=None):
        """Add a coupling constant (and optional step) for a new resonance."""
        self.couplings.append(coup)
        if step is not None:
            # If step provided, ensure couplings_steps list exists
            self.couplings_steps.append(step)

    def getCouplingSteps(self):
        """Get the list of step adjustments for couplings (if any)."""
        return list(self.couplings_steps)

    def setChebyCoeffs(self, ptype, s0, coeffs, coeffs_steps=None):
        """Set Chebyshev expansion coefficients for the channel.
        - ptype: pole type (integer) that determines the omega mapping.
        - s0: channel-specific scale parameter for omega_p mapping.
        - coeffs: list of Chebyshev coefficients.
        - coeffs_steps: optional list of step adjustments for the coefficients.
        """
        self.poletype = ptype
        self.s0 = s0
        self.chebyCoefficients = list(coeffs)
        if coeffs_steps is not None:
            self.chebyCoeff_steps = list(coeffs_steps)

    def getChebyCoeffs(self):
        """Get the list of Chebyshev coefficients."""
        return list(self.chebyCoefficients)

    def getChebyCoeff(self, i):
        """Get the i-th Chebyshev coefficient."""
        return self.chebyCoefficients[i]

    def getChebySteps(self):
        """Get the list of step adjustments for Chebyshev coefficients (if any)."""
        return list(self.chebyCoeff_steps)

    def getPoleType(self):
        """Get the pole type associated with this channel (used for omega mapping selection)."""
        return self.poletype

    def setMasses(self, masses):
        """Set the list of particle masses for this channel."""
        self.masses = list(masses)

    def setName(self, name):
        """Set the name of the channel."""
        self.channel_name = name

    def getName(self):
        """Get the name of the channel."""
        return self.channel_name


In [26]:
import torch
import math
from scipy.integrate import quad

class Amplitude:
    def __init__(self, channels, res_masses, kParameters=None, J=0, alpha=1.0, sL=0.0, smin=0.0, smax=0.0):
        """
        Initialize the Amplitude.
        :param channels: list of Channel objects.
        :param res_masses: list of resonance masses (or pole positions).
        :param kParameters: list of torch.Tensor matrices for K-matrix polynomial terms.
        :param J: angular momentum.
        :param alpha: exponent parameter for phase-space.
        :param sL: additional scale in phase-space factor.
        :param smin: minimum s for mapping.
        :param smax: maximum s for mapping.
        """
        self.channels = channels
        self.num_channels = len(channels)
        self.res_masses = res_masses
        self.kParameters = kParameters if kParameters is not None else []
        self.J = J
        self.alpha = alpha
        self.sL = sL
        self.smin = smin
        self.smax = smax
        # Default types; can be overridden after construction.
        self.kmatrix_type = "default"
        self.rhoN_type = "default"
        self.epsilon = 1e-3  # For handling singularities in integration.
        self._integral_cache = {}

    # ------------------ Helper functions ------------------
    def chebyshev(self, x, n):
        """Compute the Chebyshev polynomial T_n(x) iteratively."""
        x = x.to(torch.complex128)
        if n == 0:
            return torch.ones_like(x, dtype=torch.complex128)
        elif n == 1:
            return x
        T_nm2 = torch.ones_like(x, dtype=torch.complex128)
        T_nm1 = x
        for k in range(2, n+1):
            T_n = 2 * x * T_nm1 - T_nm2
            T_nm2, T_nm1 = T_nm1, T_n
        return T_n

    def legendreII(self, x, n):
        """Compute the Legendre function of the second kind Q_n(x) via recurrence."""
        x = x.to(torch.complex128)
        if n == 0:
            return 0.5 * torch.log((1 + x) / (1 - x))
        elif n == 1:
            return x * self.legendreII(x, 0) - 1.0
        Q_nm2 = self.legendreII(x, 0)
        Q_nm1 = self.legendreII(x, 1)
        for k in range(2, n+1):
            Q_n = ((2*k - 1)/k) * x * Q_nm1 - ((k - 1)/k) * Q_nm2
            Q_nm2, Q_nm1 = Q_nm1, Q_n
        return Q_n

    # Omega mappings:
    def omega_p(self, s):
        """omega_p(s) = s / (s + current_s0). current_s0 should be set externally."""
        s = s.to(torch.complex128) if torch.is_tensor(s) else torch.tensor(s, dtype=torch.complex128)
        return s / (s + self.current_s0)
    def omega_s(self, s):
        """omega_s(s) = 2*(s - smin)/(smax - smin) - 1."""
        s = s.to(torch.complex128) if torch.is_tensor(s) else torch.tensor(s, dtype=torch.complex128)
        return 2.0 * (s - self.smin) / (self.smax - self.smin) - 1.0
    def omega_ps(self, s):
        """omega_ps(s): Map s via omega_p, then linearly onto [-1,1] using smin and smax."""
        op = self.omega_p(s)
        op_min = self.omega_p(self.smin)
        op_max = self.omega_p(self.smax)
        return 2.0 * (op - op_min) / (op_max - op_min) - 1.0
    def omega(self, s, type):
        """Select omega mapping based on type (1 -> omega_p, 2 -> omega_s, 3 -> omega_ps)."""
        if type == 1:
            return self.omega_p(s)
        elif type == 2:
            return self.omega_s(s)
        elif type == 3:
            return self.omega_ps(s)
        return s

    # ------------------ Numerator ------------------
    def getNumerator(self, s, type_override=None):
        """
        Compute the numerator vector N(s) for all channels.
        For each channel k: N_k(s) = sum_n [a_n * T_n(omega(s))],
        where a_n are the Chebyshev coefficients.
        :param s: energy squared.
        :param type_override: if provided, use this mapping type for all channels.
        :return: torch.complex128 tensor of shape (num_channels,)
        """
        s = s.to(torch.complex128) if torch.is_tensor(s) else torch.tensor(s, dtype=torch.complex128)
        numerator = torch.zeros(self.num_channels, dtype=torch.complex128)
        for k, channel in enumerate(self.channels):
            pole_type = type_override if type_override is not None else channel.pole_type
            # For omega_p mapping, use channel-specific pivot stored in channel.cheby_s0
            if pole_type == 1:
                self.current_s0 = channel.cheby_s0
            else:
                self.current_s0 = 1.0
            omega_val = self.omega(s, pole_type)
            n_val = torch.zeros((), dtype=torch.complex128)
            coeffs = channel.cheby_coeffs if hasattr(channel, 'cheby_coeffs') else []
            if len(coeffs) == 0:
                numerator[k] = 0.0
                continue
            T0 = torch.ones_like(omega_val, dtype=torch.complex128)
            T1 = omega_val
            n_val += coeffs[0] * T0
            if len(coeffs) > 1:
                n_val += coeffs[1] * T1
            T_prev2 = T0
            T_prev1 = T1
            for n in range(2, len(coeffs)):
                T_curr = 2 * omega_val * T_prev1 - T_prev2
                n_val += coeffs[n] * T_curr
                T_prev2, T_prev1 = T_prev1, T_curr
            numerator[k] = n_val
        return numerator

    # ------------------ K-matrix ------------------
    def getKMatrix(self, s):
        """
        Compute the K-matrix K(s) as an N x N complex matrix.
        Includes resonance pole contributions and optional polynomial terms.
        """
        s_t = s.to(torch.complex128) if torch.is_tensor(s) else torch.tensor(s, dtype=torch.complex128)
        N = self.num_channels
        K_mat = torch.zeros((N, N), dtype=torch.complex128)
        for i in range(N):
            for j in range(i, N):
                term = 0+0j
                for R, mR in enumerate(self.res_masses):
                    g_iR = self.channels[i].getCoupling(R) if R < len(self.channels[i].getCouplings()) else 0.0
                    g_jR = self.channels[j].getCoupling(R) if R < len(self.channels[j].getCouplings()) else 0.0
                    term += g_iR * g_jR / (mR - s_t)
                if isinstance(term, torch.Tensor):
                    K_mat[i, j] = term.clone().detach()
                else:
                    K_mat[i, j] = torch.tensor(term, dtype=torch.complex128)
                K_mat[j, i] = K_mat[i, j]
        if self.kParameters:
            for j, k_param in enumerate(self.kParameters):
                k_param = k_param.to(torch.complex128)
                if self.kmatrix_type == "kmat-CDD":
                    K_mat = K_mat + ((-s_t) ** j) * k_param
                else:
                    K_mat = K_mat + (s_t ** j) * k_param
        return K_mat

    # ------------------ Dispersive Integral ------------------
    def getIntegral(self, s, k: int, sheet: int = 0) -> torch.Tensor:
        """
        Compute the dispersive integral I_k(s) for channel k on the specified sheet.
        Caches computed values to avoid redundant calculations.
        """
        s_val = complex(s) if not torch.is_tensor(s) else complex(s.item())
        cache_key = (sheet, k, s_val)
        if cache_key in self._integral_cache:
            return self._integral_cache[cache_key]
        result = 0+0j
        sh_flag = bool(((2 ** sheet) - 1) & (2 ** k))
        if sh_flag:
            result += 2.0 * self.get_rhoN(s, k, sheet)
        threshold = self.channels[k].getThreshold()
        if s_val.real <= threshold:
            tempRhoN = 0.0
        else:
            p_val = 0.5 * math.sqrt(max(s_val.real - threshold, 0.0))
            tempRhoN = (2.0 * p_val) ** (2 * self.J + 1) / ((s_val + self.sL) ** (self.J + self.alpha))
        def integrand(sp):
            sp_val = sp
            if sp_val <= threshold:
                rho_sp = 0.0
            else:
                p_val = 0.5 * math.sqrt(max(sp_val - threshold, 0.0))
                rho_sp = (2.0 * p_val) ** (2 * self.J + 1) / ((sp_val + self.sL) ** (self.J + self.alpha))
            return (s_val * (rho_sp - tempRhoN) / (sp_val * (sp_val - s_val))) / math.pi
        try:
            real_part, _ = quad(integrand, threshold + self.epsilon, math.inf, limit=100, epsabs=1e-6, epsrel=1e-6)
        except Exception as e:
            real_part = 0.0
        try:
            log_term = tempRhoN * math.log(threshold / abs(s_val - threshold)) / math.pi
        except ValueError:
            log_term = 0.0
        imag_correction = 1j * tempRhoN if s_val.real > threshold else 0.0
        result += real_part + log_term + imag_correction
        res_tensor = torch.tensor(result, dtype=torch.complex128)
        self._integral_cache[cache_key] = res_tensor
        return res_tensor

    # ------------------ Phase-space Factor (rho_N) ------------------
    def get_rhoN(self, sprime, k: int, sheet: int = 0) -> torch.Tensor:
        """
        Compute the phase-space factor rho_N for channel k at energy sprime on the specified sheet.
        Default: rho(s') = (2p)^(2J+1) / (s'+sL)^(J+alpha).
        """
        sprime = sprime.to(torch.complex128) if torch.is_tensor(sprime) else torch.tensor(sprime, dtype=torch.complex128)
        sh_flag = bool(((2 ** sheet) - 1) & (2 ** k))
        J = self.J
        p = self.channels[k].getMomentum(sprime)
        rho = (2.0 * p) ** (2 * J + 1) / ((sprime + self.sL) ** (J + self.alpha))
        if sh_flag:
            p_c = self.channels[k].getComplexMomentum(sprime)
            rho = ((-1.0) ** J) * (2.0 * p_c) ** (2 * J + 1) / ((sprime + self.sL) ** (J + self.alpha))
        return rho

    # ------------------ Evaluate Amplitude ------------------
    def getValue(self, s):
        """
        Evaluate the multi-channel amplitude A(s) at energy squared s.
        Computes A(s) = N^T * (I - K * M)^{-1} * K * P, where:
          - N is the numerator vector,
          - K is the K-matrix,
          - M is the diagonal matrix of dispersive integrals I_k(s),
          - P is the phase-space diagonal matrix with P_ii = (p_i(s))^(J+0.5) / s^(0.25).
        :param s: energy squared (s)
        :return: torch.complex128 tensor of shape (num_channels,)
        """
        s_t = s.to(torch.complex128) if torch.is_tensor(s) else torch.tensor(s, dtype=torch.complex128)
        N = self.getNumerator(s_t, type_override=None)
        K_mat = self.getKMatrix(s_t)
        I_mat = torch.eye(self.num_channels, dtype=torch.complex128)
        DispRhoN = torch.zeros((self.num_channels, self.num_channels), dtype=torch.complex128)
        for k in range(self.num_channels):
            DispRhoN[k, k] = self.getIntegral(s_t, k, sheet=0)
        try:
            denom_inv = torch.linalg.inv(I_mat - K_mat @ DispRhoN)
        except Exception as e:
            print("Error in inverting (I - K*DispRhoN):", e)
            raise e
        # Phase-space diagonal matrix P:
        P = torch.eye(self.num_channels, dtype=torch.complex128)
        for i in range(self.num_channels):
            p_true = self.channels[i].getTrueMomentum(s_t)
            P[i, i] = (p_true ** (self.J + 0.5)) / (s_t ** 0.25)
        # Compute amplitude: A = N^T * denom_inv * K * diag(P)
        A_vec = (self.getNumerator(s_t, type_override=None).view(1, -1) @ denom_inv @ K_mat)
        # Multiply element-wise by phase-space factors (from the diagonal of P)
        A_vec = A_vec * P.diag()
        return A_vec.view(-1)

In [30]:
    # Example Channel objects (ensure these methods exist in your Channel class)
    chan1 = Channel("Channel1", masses=[0.5, 0.5], couplings=[1.2, 0.8])
    chan1.cheby_coeffs = [1.0, 0.5, 0.2]
    chan1.cheby_s0 = 1.0
    chan1.pole_type = 1

    chan2 = Channel("Channel2", masses=[1.0, 0.3], couplings=[0.5, 0.1])
    chan2.cheby_coeffs = [0.2, 0.1]
    chan2.cheby_s0 = 2.0
    chan2.pole_type = 1

    res_masses = [1.5]
    k_params = [torch.zeros((2, 2))]
    amp = Amplitude(channels=[chan1, chan2], res_masses=res_masses, kParameters=k_params,
                    J=0, alpha=1.0, sL=0.0, smin=0.0, smax=5.0)
    amp.kmatrix_type = "default"
    amp.rhoN_type = "default"

    s_val = 2.0
    A = amp.getValue(s_val)
    print(f"Amplitude at s={s_val}: {A}")

Amplitude at s=2.0: tensor([-0.6003+0.8818j, -0.1740+0.2556j], dtype=torch.complex128)


In [31]:
import re

class FileReader:
    def __init__(self, filename: str):
        """Read the given file and parse commands to create Amplitude and Channel objects."""
        # Containers for parsed objects
        self.channels = {}    # e.g., {"Name": Channel(...), ...}
        self.amplitudes = {}  # e.g., {"WaveName": Amplitude(...), ...}
        self.smin = None
        self.smax = None

        # Open and read all lines from the file
        with open(filename, 'r') as infile:
            lines = [line.strip() for line in infile if line.strip() != ""]

        # 1. Parse FitRegion (if present) to get smin and smax
        for line in lines:
            if line.startswith("FitRegion"):
                # Example: FitRegion(0.9975, 2.5)
                content = line[line.find('(')+1 : line.rfind(')')]
                parts = [p.strip() for p in content.split(',')]
                if len(parts) >= 2:
                    # Convert the two numbers to float and square them
                    x_val = float(parts[0])
                    y_val = float(parts[1])
                    self.smin = x_val ** 2
                    self.smax = y_val ** 2
                break  # assume only one FitRegion command

        # 2. Parse AddChannel commands to create Channel objects
        for line in lines:
            if line.startswith("AddChannel"):
                # Format: AddChannel("Name", {m1, m2})
                content = line[line.find('(')+1 : line.rfind(')')]  # inside the parentheses
                # Split into name part and mass part at the first comma
                comma_idx = content.find(',')
                if comma_idx == -1:
                    continue  # should not happen if input is well-formed
                name_part = content[:comma_idx].strip()
                masses_part = content[comma_idx+1:].strip()
                # Extract channel name (remove quotes)
                if name_part.startswith('"') and name_part.endswith('"'):
                    chan_name = name_part[1:-1]
                else:
                    chan_name = name_part
                chan_name = chan_name.replace(" ", "")  # remove any spaces in name
                # Extract masses list (remove braces and split by comma)
                if masses_part.startswith('{') and masses_part.endswith('}'):
                    masses_str = masses_part[1:-1]
                else:
                    masses_str = masses_part
                mass_values = [float(x.strip()) for x in masses_str.split(',') if x.strip()]
                # Create Channel object and store it
                new_channel = Channel(chan_name, mass_values)
                self.channels[chan_name] = new_channel

        # 3. Parse AddWave commands to create Amplitude objects
        for line in lines:
            if line.startswith("AddWave"):
                # Format: AddWave("WaveName", "kmatFlag", "rhoNFlag", J, sL)
                content = line[line.find('(')+1 : line.rfind(')')]
                # Use regex to capture components for reliability
                pattern = r'^AddWave\(\s*"(?P<wname>.*?)"\s*,\s*"(?P<kmat>.*?)"\s*,\s*"(?P<rhoN>.*?)"\s*,\s*(?P<J>[0-9\.-]+)\s*,\s*(?P<sL>[0-9\.-]+)\s*\)$'
                match = re.match(pattern, line)
                if match:
                    wname = match.group('wname').replace(" ", "")
                    kmat_flag = match.group('kmat')
                    rhoN_flag = match.group('rhoN')
                    J_val = int(float(match.group('J')))  # convert J to int
                    sL_val = float(match.group('sL'))
                else:
                    # Fallback parsing if regex doesn't match (split by commas outside quotes)
                    parts = []
                    current = ""
                    in_quotes = False
                    for ch in content:
                        if ch == '"' and not in_quotes:
                            in_quotes = True
                            current += ch
                        elif ch == '"' and in_quotes:
                            current += ch
                            in_quotes = False
                        elif ch == ',' and not in_quotes:
                            parts.append(current.strip())
                            current = ""
                        else:
                            current += ch
                    if current:
                        parts.append(current.strip())
                    # parts should now contain ['"Name"', '"kmat"', '"rhoN"', 'J', 'sL']
                    wname = parts[0].strip('"').replace(" ", "")
                    kmat_flag = parts[1].strip('"')
                    rhoN_flag = parts[2].strip('"')
                    J_val = int(float(parts[3])) if parts[3] else 0
                    sL_val = float(parts[4]) if parts[4] else 0.0
                # Prepare channel list for this amplitude (make new instances to avoid shared state)
                channel_list = [Channel(ch.name, ch.masses) for ch in self.channels.values()]
                # Create Amplitude object with name, J, sL, smin/smax (if FitRegion was provided), and channel list
                amp_name = wname  # already stripped of spaces
                new_amp = Amplitude(amp_name, J_val, sL_val, 
                                     self.smin if self.smin is not None else 0.0,
                                     self.smax if self.smax is not None else 0.0,
                                     channel_list, kmat_flag, rhoN_flag)
                self.amplitudes[amp_name] = new_amp

        # 4. Parse ChebyCoeffs commands to set Chebyshev coefficients
        for line in lines:
            if line.startswith("ChebyCoeffs"):
                # Format: ChebyCoeffs("Wave", "Channel", "ptype s0", {coeff ± δ, coeff ± δ, ...})
                content = line[line.find('(')+1 : line.rfind(')')]
                # Separate the arguments part (up to the coefficients list) and the coefficients list itself
                coeff_start = content.find('{')
                args_str = content[:coeff_start].rstrip(', ')
                coeffs_str = content[coeff_start+1 : -1]  # inside the braces of coefficients
                # Split the arguments by comma (the first three arguments are quoted strings)
                args = [arg.strip() for arg in args_str.split(',') if arg.strip()]
                if len(args) < 3:
                    continue  # not a valid ChebyCoeffs line
                # Extract amplitude name, channel name, and pole type string
                wavename = args[0].strip('"').replace(" ", "")
                chan_name = args[1].strip('"').replace(" ", "")
                type_and_s0 = args[2].strip('"')
                # Determine pole type code and s0 value
                parts = type_and_s0.split()
                ptype_str = parts[0]
                s0_val = float(parts[1]) if len(parts) > 1 else 0.0
                pole_type = 0
                if ptype_str == "p":
                    pole_type = 1
                elif ptype_str == "s":
                    pole_type = 2
                elif ptype_str == "p+s":
                    pole_type = 3
                # Parse coefficients and their increments from coeffs_str
                coeff_values = []
                coeff_increments = []
                for item in coeffs_str.split(','):
                    item = item.strip()
                    if not item:
                        continue
                    # Each item is like "a ± da"
                    if '±' in item:
                        val_str, inc_str = item.split('±')
                    elif '+-' in item:
                        val_str, inc_str = item.split('+-')
                    elif 'pm' in item:  # in case \pm was used
                        val_str, inc_str = item.split('pm')
                    else:
                        val_str, inc_str = item, '0'
                    coeff_values.append(float(val_str.strip()))
                    coeff_increments.append(float(inc_str.strip()))
                # Set Chebyshev coefficients on the corresponding amplitude
                if wavename in self.amplitudes:
                    amp_obj = self.amplitudes[wavename]
                    amp_obj.setChebyCoeffs(chan_name, pole_type, s0_val, coeff_values, coeff_increments)

        # 5. Parse AddPole commands to add resonance poles
        for line in lines:
            if line.startswith("AddPole"):
                # Format: AddPole("Wave", M ± dM, {"Ch1", "Ch2", ...}, {g1 ± dg1, g2 ± dg2, ...})
                content = line[line.find('(')+1 : line.rfind(')')]
                # Extract amplitude name
                pos1 = content.find('"')
                pos2 = content.find('"', pos1+1)
                if pos1 == -1 or pos2 == -1:
                    continue  # malformed line
                wavename = content[pos1+1:pos2].replace(" ", "")
                # Extract the mass and mass uncertainty (between first and second comma)
                comma1 = content.find(',', pos2)         # comma after amplitude name
                comma2 = content.find(',', comma1+1)     # comma after mass specification
                mass_str = content[comma1+1 : comma2].strip()
                if '±' in mass_str:
                    m_val_str, m_inc_str = mass_str.split('±')
                elif '+-' in mass_str:
                    m_val_str, m_inc_str = mass_str.split('+-')
                elif 'pm' in mass_str:
                    m_val_str, m_inc_str = mass_str.split('pm')
                else:
                    m_val_str, m_inc_str = mass_str, '0'
                mass_val = float(m_val_str.strip())
                mass_inc = float(m_inc_str.strip())
                # Extract channel names list (between the first set of braces)
                chan_list_start = content.find('{', comma2)
                chan_list_end = None
                balance = 0
                for i in range(chan_list_start, len(content)):
                    if content[i] == '{':
                        balance += 1
                    elif content[i] == '}':
                        balance -= 1
                        if balance == 0:
                            chan_list_end = i
                            break
                if chan_list_start == -1 or chan_list_end is None:
                    continue  # malformed channel list
                chan_list_content = content[chan_list_start+1 : chan_list_end]
                # Find all channel names inside the braces (they are quoted and separated by commas)
                channel_names = re.findall(r'"([^"]+)"', chan_list_content)
                channel_names = [name.replace(" ", "") for name in channel_names]
                # Extract coupling values list (between the second set of braces)
                coup_list_start = content.find('{', chan_list_end)
                coup_list_end = content.find('}', coup_list_start)
                if coup_list_start == -1 or coup_list_end == -1:
                    continue  # malformed coupling list
                coup_list_content = content[coup_list_start+1 : coup_list_end]
                # Parse each coupling value and its uncertainty
                couplings = []
                coupling_incs = []
                for item in coup_list_content.split(','):
                    item = item.strip()
                    if not item:
                        continue
                    if '±' in item:
                        val_str, inc_str = item.split('±')
                    elif '+-' in item:
                        val_str, inc_str = item.split('+-')
                    elif 'pm' in item:
                        val_str, inc_str = item.split('pm')
                    else:
                        val_str, inc_str = item, '0'
                    couplings.append(float(val_str.strip()))
                    coupling_incs.append(float(inc_str.strip()))
                # Call addPole on the corresponding Amplitude
                if wavename in self.amplitudes:
                    amp_obj = self.amplitudes[wavename]
                    amp_obj.addPole(mass_val, mass_inc, channel_names, couplings, coupling_incs)

        # 6. Parse AddKmatBackground commands to set K-matrix parameters
        for line in lines:
            if line.startswith("AddKmatBackground"):
                # Format: AddKmatBackground("Wave", N, {{x ± dx, y ± dy, ...}, {...}, ...})
                content = line[line.find('(')+1 : line.rfind(')')]
                # Extract amplitude name
                pos1 = content.find('"')
                pos2 = content.find('"', pos1+1)
                if pos1 == -1 or pos2 == -1:
                    continue
                wavename = content[pos1+1:pos2].replace(" ", "")
                # Extract the power (between first comma after name and the next comma)
                comma1 = content.find(',', pos2)
                comma2 = content.find(',', comma1+1)
                power_str = content[comma1+1 : comma2].strip()
                if not power_str:
                    continue
                power = int(float(power_str))
                # Extract the matrix elements inside the outer braces
                matrix_start = content.find('{', comma2)
                matrix_end = None
                balance = 0
                for i in range(matrix_start, len(content)):
                    if content[i] == '{':
                        balance += 1
                    elif content[i] == '}':
                        balance -= 1
                        if balance == 0:
                            matrix_end = i
                            break
                if matrix_start == -1 or matrix_end is None:
                    continue  # malformed matrix block
                matrix_content = content[matrix_start+1 : matrix_end]  # drop outer braces
                # Parse each row of the matrix (each row is enclosed in braces)
                rows = []
                steps = []
                idx = 0
                while idx < len(matrix_content):
                    # Find the next row's braces
                    row_start = matrix_content.find('{', idx)
                    if row_start == -1:
                        break
                    balance2 = 0
                    row_end = None
                    for j in range(row_start, len(matrix_content)):
                        if matrix_content[j] == '{':
                            balance2 += 1
                        elif matrix_content[j] == '}':
                            balance2 -= 1
                            if balance2 == 0:
                                row_end = j
                                break
                    if row_end is None:
                        break
                    row_str = matrix_content[row_start+1 : row_end]
                    # Parse numbers in this row
                    values = []
                    inc_values = []
                    for item in row_str.split(','):
                        item = item.strip()
                        if not item:
                            continue
                        if '±' in item:
                            val_str, inc_str = item.split('±')
                        elif '+-' in item:
                            val_str, inc_str = item.split('+-')
                        elif 'pm' in item:
                            val_str, inc_str = item.split('pm')
                        else:
                            val_str, inc_str = item, '0'
                        values.append(float(val_str.strip()))
                        inc_values.append(float(inc_str.strip()))
                    rows.append(values)
                    steps.extend(inc_values)
                    # Move index to character after this row to find subsequent rows
                    idx = row_end + 1
                # Set K-matrix parameters on the amplitude
                if wavename in self.amplitudes:
                    amp_obj = self.amplitudes[wavename]
                    amp_obj.setKParams(power, rows, steps)
